# RegNet Inference (submission generation)

In [87]:
import os
import sys
from pathlib import Path
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import numpy as np
from torchvision.models import regnet_y_800mf, RegNet_Y_800MF_Weights
import timm

In [88]:
## REUSE IN INFERENCE NOTEBOOK

custom_dataset_path = '/kaggle/input/birdclef2023-inference'
if os.path.exists(os.path.join(custom_dataset_path, 'utils.py')):
    sys.path.append(custom_dataset_path)
else:
    sys.path.append('..')
import utils

IS_IN_KAGGLE_ENV = utils.get_is_in_kaggle_env()

DATA_PATH = '/kaggle/input/birdclef-2023' if IS_IN_KAGGLE_ENV else '../data'
JOBLIB_PATH = custom_dataset_path if IS_IN_KAGGLE_ENV else './'

DEVICE = 'cpu'

AUDIO_LENGTH_S = 5
SAMPLE_RATE = 32_000

We are running code on Localhost


In [89]:
## REUSE IN INFERENCE NOTEBOOK

# class BirdMelspecClf(nn.Module):
#     def __init__(self, out_features, pretrained):
#         super().__init__()

#         self.cnn = timm.create_model('regnety_080', pretrained=pretrained)
#         for name, param in self.cnn.named_parameters():
#             if param.requires_grad:
#                 print(name)

#         # Replace original stem for black and white images
#         self.cnn.stem.conv = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

#         # Replace original classifier for our task
#         self.cnn.head.fc = nn.Sequential(
#             nn.Linear(self.cnn.head.fc.in_features, 1024),
#             nn.BatchNorm1d(1024),
#             nn.PReLU(),

#             nn.Linear(1024, 512),
#             nn.BatchNorm1d(512),
#             nn.PReLU(),
            
#             nn.Linear(512, out_features),
#         )

#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         logits = self.cnn(x)
#         probas = self.softmax(logits)

#         return logits, probas


class BirdMelspecClf(nn.Module):
    def __init__(self, out_features, pretrained):
        super().__init__()
        
        # https://pytorch.org/vision/stable/models.html

        self.regnet = regnet_y_800mf(weights=RegNet_Y_800MF_Weights.DEFAULT) if pretrained else regnet_y_800mf()

        """
        Replace the stem to take 1 channel instead of 3. The original stem:
        RegnetCNN(
        (regnet): RegNet(
            (stem): SimpleStemIN(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
        )"""
        self.regnet.stem = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
        )
        
        # Fine-tune the regnet classifier
        self.regnet.fc = nn.Sequential(
            nn.Linear(self.regnet.fc.in_features, 512),
            nn.BatchNorm1d(512),
            nn.PReLU(),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            
            nn.Linear(256, out_features),
        )

        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, x):
        logits = self.regnet(x)
        probas = self.softmax(logits)

        return logits, probas


def get_model(out_features, device, pretrained=False, load_state_dict=True):
    model = BirdMelspecClf(out_features=out_features, pretrained=pretrained)
    print(f"Loaded model {model.__class__.__name__} with {sum(p.numel() for p in model.parameters())} parameters, pretained={pretrained}")
    model.to(device)
    if load_state_dict:
        model_path = os.path.join(JOBLIB_PATH, 'best_model.pt')
        model.load_state_dict(torch.load(model_path))
        print(f"Loaded model weights from {model_path}")
    model.to(device)
    return model


def get_label_encoder():
    label_encoder_path = os.path.join(JOBLIB_PATH, 'label_encoder.joblib')
    label_encoder = joblib.load(label_encoder_path)
    print(f"Loaded label encoder from {label_encoder_path}")
    return label_encoder

In [90]:
## REUSE IN INFERENCE NOTEBOOK

def resample(audio, current_sample_rate, desired_sample_rate=SAMPLE_RATE):
    resampler = torchaudio.transforms.Resample(orig_freq=current_sample_rate, new_freq=desired_sample_rate)
    resampled_audio = resampler(audio)
    return resampled_audio

def load_audio(audio_path, sample_rate=SAMPLE_RATE):
    audio, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        audio = resample(audio, sr, sample_rate)
    return audio

def get_melspec_transform(sample_rate=SAMPLE_RATE, n_fft=1024, win_length=1024, hop_length=512, n_mels=128, f_min=0, f_max=SAMPLE_RATE // 2, normalized=True):
    return torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        n_mels=n_mels,
        f_min=f_min,
        f_max=f_max,
        normalized=normalized,
    )

def get_melspec_db_transform(stype='power', top_db=80):
    return torchaudio.transforms.AmplitudeToDB(
        stype=stype,
        top_db=top_db
    )

def normalize_melspec(spectrogram):
    min_val = torch.min(spectrogram)
    max_val = torch.max(spectrogram)
    if max_val - min_val == 0:
        return spectrogram
    else:
        return (spectrogram - min_val) / (max_val - min_val)

In [91]:
filepaths = glob.glob(f"{DATA_PATH}/test_soundscapes/*.ogg")
print(f"filepaths length: {len(filepaths)} (amount of test soundscapes)")

filepaths length: 1 (amount of test soundscapes)


# Inference

In [92]:
debug = False
simulate_200_files = False

if simulate_200_files:
    filepaths = [filepaths[0] for i in range(200)] # simulate submission
    print(f"filepaths length: {len(filepaths)} after simulation additions")

label_encoder = get_label_encoder()
model = get_model(out_features=len(label_encoder.classes_), device=DEVICE, pretrained=False, load_state_dict=True)
model.eval()

MIN_WINDOW = AUDIO_LENGTH_S * SAMPLE_RATE
melspec_transform = get_melspec_transform(n_mels=128)
melspec_db_transform = get_melspec_db_transform()

def infer(filepath):
    all_predictions = []
    name = Path(filepath).stem
    audio = torchaudio.load(filepath)[0][0]
    audio_len_s = len(audio) / SAMPLE_RATE
    debug and print(f"Infering file {filepath} with length {audio_len_s} s")
    n_crops = int(audio_len_s // 5)
    for i in range(n_crops):
        debug and print(f"Crop {i} / {n_crops}")
        debug and print(f"Audio length: {len(audio)}")
        crop = audio[i*MIN_WINDOW:(i+1)*MIN_WINDOW]
        debug and print(f"Crop length: {len(crop)}")
        melspec = normalize_melspec(melspec_db_transform(melspec_transform(crop)))
        debug and print(f"melspec shape: {melspec.shape}") # [128, 313]
        melspec = melspec.unsqueeze(0) # add batch dimension (1)
        debug and print(f"melspec unsqueezed shape: {melspec.shape}") # [1, 128, 313]
        melspec = melspec.unsqueeze(0) # add channel dimension (1)
        debug and print(f"melspec unsqueezed 2 shape: {melspec.shape}") # [1, 1, 128, 313]
        with torch.no_grad():
            logit, proba = model(melspec)
        t = (i + 1) * 5
        all_predictions.append({"row_id": f'{name}_{t}',"predictions": proba})
        debug and print('---')
    return all_predictions

if debug:
    all_preds = []
    for filepath in tqdm(filepaths, desc='Infering files'):
        all_preds.append(infer(filepath))
else:
    parallel_task = (delayed(infer)(filepath) for filepath in tqdm(filepaths, desc='Infering files'))
    all_preds = Parallel(n_jobs=os.cpu_count())(parallel_task)

all_preds_flat = [item for sublist in all_preds for item in sublist]

print(f"all_preds length: {len(all_preds)}, all_preds_flat length: {len(all_preds_flat)}")

Loaded label encoder from ./label_encoder.joblib
Loaded model BirdMelspecClf with 6249570 parameters, pretained=False
Loaded model weights from ./best_model.pt


Infering files: 100%|██████████| 1/1 [00:00<00:00, 1270.62it/s]


all_preds length: 1, all_preds_flat length: 120


In [93]:
all_preds_flat[100]['predictions'][0]

tensor([[0.0023, 0.0043, 0.0046, 0.0022, 0.0049, 0.0067, 0.0033, 0.0024, 0.0039,
         0.0043, 0.0061, 0.0016, 0.0073, 0.0035, 0.0017, 0.0042, 0.0038, 0.0026,
         0.0066, 0.0035, 0.0073, 0.0046, 0.0017, 0.0037, 0.0019, 0.0033, 0.0047,
         0.0046, 0.0057, 0.0028, 0.0052, 0.0054, 0.0036, 0.0031, 0.0026, 0.0020,
         0.0043, 0.0011, 0.0014, 0.0027, 0.0039, 0.0061, 0.0038, 0.0015, 0.0024,
         0.0017, 0.0016, 0.0027, 0.0040, 0.0031, 0.0021, 0.0014, 0.0067, 0.0034,
         0.0018, 0.0015, 0.0056, 0.0018, 0.0025, 0.0037, 0.0044, 0.0034, 0.0032,
         0.0040, 0.0012, 0.0010, 0.0033, 0.0039, 0.0044, 0.0034, 0.0036, 0.0052,
         0.0071, 0.0059, 0.0051, 0.0017, 0.0028, 0.0026, 0.0087, 0.0024, 0.0078,
         0.0033, 0.0041, 0.0022, 0.0064, 0.0044, 0.0029, 0.0010, 0.0051, 0.0045,
         0.0063, 0.0018, 0.0025, 0.0051, 0.0029, 0.0057, 0.0035, 0.0062, 0.0037,
         0.0057, 0.0037, 0.0017, 0.0011, 0.0035, 0.0024, 0.0063, 0.0056, 0.0069,
         0.0037, 0.0077, 0.0

In [94]:
df = pd.concat([
    pd.DataFrame({'row_id': [p['row_id'] for p in all_preds_flat]}), 
    pd.DataFrame(torch.stack([p['predictions'][0] for p in all_preds_flat]).numpy(), columns=label_encoder.classes_)
], axis=1)

df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.002553,0.007871,0.002371,0.003698,0.007275,0.004447,0.004649,0.001115,0.002954,...,0.000958,0.002049,0.003065,0.001053,0.001555,0.006383,0.004461,0.005751,0.005373,0.002764
1,soundscape_29201_10,0.001526,0.005301,0.003628,0.002676,0.005660,0.003175,0.006224,0.001529,0.003261,...,0.000446,0.002153,0.004075,0.000854,0.000966,0.002369,0.003578,0.004858,0.004662,0.005437
2,soundscape_29201_15,0.001910,0.010668,0.001762,0.003290,0.007201,0.004961,0.004633,0.001756,0.003506,...,0.001584,0.001713,0.005357,0.001498,0.002184,0.005069,0.003024,0.003996,0.005156,0.005030
3,soundscape_29201_20,0.001272,0.011509,0.002380,0.004180,0.005241,0.004470,0.003826,0.001109,0.001737,...,0.001149,0.002233,0.003014,0.001398,0.001273,0.004108,0.003038,0.004974,0.003973,0.007079
4,soundscape_29201_25,0.002244,0.007616,0.002314,0.001992,0.004259,0.002761,0.003530,0.001655,0.003771,...,0.000719,0.001536,0.001645,0.001016,0.001259,0.004099,0.003172,0.004596,0.002280,0.006741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.002585,0.005210,0.002640,0.003660,0.004738,0.002888,0.003610,0.001867,0.004097,...,0.000841,0.002024,0.006184,0.001582,0.001192,0.002057,0.001983,0.003984,0.003991,0.003851
116,soundscape_29201_585,0.002497,0.005471,0.002968,0.001779,0.007725,0.003958,0.004111,0.002175,0.002732,...,0.000950,0.002861,0.004219,0.001444,0.001270,0.004271,0.002731,0.005446,0.004751,0.003437
117,soundscape_29201_590,0.002431,0.004588,0.005720,0.002030,0.005555,0.005315,0.003433,0.002565,0.004387,...,0.001112,0.002015,0.003731,0.001382,0.001863,0.002197,0.006212,0.006483,0.007140,0.005190
118,soundscape_29201_595,0.001825,0.007921,0.004220,0.003338,0.004572,0.003841,0.002829,0.001289,0.004766,...,0.001039,0.001740,0.006645,0.001784,0.002158,0.002918,0.002676,0.004521,0.003083,0.007449


In [95]:
df.to_csv('submission.csv', index=False)

In [96]:
!ls submission.csv

submission.csv
